## 1. Installation and Setup <a name="installation"></a>

Install NeuralMOVES directly from GitHub:

In [ ]:
# Uncomment to install
# !pip install "git+https://github.com/edgar-rs/neuralMOVES.git@main"

In [ ]:
import neuralmoves
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print(f"NeuralMOVES version: {neuralmoves.__version__}")

## 2. Single-Second Estimation <a name="single-second"></a>

Estimate emissions for a single driving condition:

In [ ]:
# Estimate CO2 for a single second
emission = neuralmoves.estimate_running_co2(
    v_ms=15.0,              # 15 m/s (≈54 km/h or 33 mph)
    a_mps2=0.5,             # 0.5 m/s² acceleration
    grade_pct=0.0,          # flat road
    temp=25,                # 25°C
    temp_unit='C',
    humid_pct=50,           # 50% relative humidity
    model_year=2020,
    source_type='Passenger Car',
    fuel_type='Gasoline'
)

print(f"Estimated CO2 emission: {emission:.2f} g/s")
print(f"Over 1 minute: {emission * 60:.2f} g")
print(f"Over 1 hour: {emission * 3600:.2f} g = {emission * 3600 / 1000:.2f} kg")

### Compare idling vs. moving emissions

In [ ]:
# Get idling emission rate
idling_rate = neuralmoves.idling_rate(
    model_year=2020,
    source_type='Passenger Car',
    fuel_type='Gasoline'
)

# Compare with moving emission
moving_emission = neuralmoves.estimate_running_co2(
    v_ms=10.0, a_mps2=0.0, grade_pct=0.0,
    temp=25, temp_unit='C', humid_pct=50,
    model_year=2020, source_type='Passenger Car', fuel_type='Gasoline'
)

print(f"Idling: {idling_rate:.3f} g/s")
print(f"Moving at 10 m/s: {moving_emission:.3f} g/s")
print(f"Ratio (moving/idling): {moving_emission/idling_rate:.2f}x")

## 3. Time-Series / Driving Cycle Analysis <a name="time-series"></a>

Analyze emissions over a complete driving cycle:

In [ ]:
# Create a realistic urban driving cycle
np.random.seed(42)
time_steps = 300  # 5 minutes

# Simulate stop-and-go traffic
speeds = []
for _ in range(5):  # 5 cycles of stop-and-go
    # Acceleration phase
    speeds.extend(np.linspace(0, 15, 20))  
    # Cruising
    speeds.extend([15] * 20)
    # Deceleration
    speeds.extend(np.linspace(15, 0, 20))

speeds = np.array(speeds[:time_steps])

# Calculate accelerations
accelerations = np.diff(speeds, prepend=speeds[0])

# Create driving cycle DataFrame
driving_cycle = pd.DataFrame({
    'time_s': range(time_steps),
    'speed_ms': speeds,
    'acceleration_mps2': accelerations,
    'grade_pct': 0.0  # flat road
})

# Calculate emissions for each second
emissions = []
for _, row in driving_cycle.iterrows():
    em = neuralmoves.estimate_running_co2(
        v_ms=row['speed_ms'],
        a_mps2=row['acceleration_mps2'],
        grade_pct=row['grade_pct'],
        temp=25, temp_unit='C', humid_pct=50,
        model_year=2020,
        source_type='Passenger Car',
        fuel_type='Gasoline'
    )
    emissions.append(em)

driving_cycle['co2_gs'] = emissions
driving_cycle['co2_cumulative_g'] = driving_cycle['co2_gs'].cumsum()

print(f"Total CO2 over cycle: {driving_cycle['co2_gs'].sum():.2f} g")
print(f"Average CO2 rate: {driving_cycle['co2_gs'].mean():.2f} g/s")
print(f"Distance traveled: {driving_cycle['speed_ms'].sum():.0f} m = {driving_cycle['speed_ms'].sum()/1000:.2f} km")
print(f"CO2 per km: {driving_cycle['co2_gs'].sum() / (driving_cycle['speed_ms'].sum()/1000):.2f} g/km")

In [ ]:
# Plot the driving cycle and emissions
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Speed profile
axes[0].plot(driving_cycle['time_s'], driving_cycle['speed_ms'], 'b-', linewidth=1.5)
axes[0].set_ylabel('Speed (m/s)', fontsize=12)
axes[0].set_title('Urban Driving Cycle Analysis', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Instantaneous emissions
axes[1].plot(driving_cycle['time_s'], driving_cycle['co2_gs'], 'r-', linewidth=1.5)
axes[1].set_ylabel('CO₂ Emission Rate (g/s)', fontsize=12)
axes[1].grid(True, alpha=0.3)

# Cumulative emissions
axes[2].plot(driving_cycle['time_s'], driving_cycle['co2_cumulative_g'], 'g-', linewidth=2)
axes[2].set_xlabel('Time (s)', fontsize=12)
axes[2].set_ylabel('Cumulative CO₂ (g)', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Multi-Vehicle Comparison <a name="multi-vehicle"></a>

Compare emissions across different vehicle types and fuel types:

In [ ]:
# Define test conditions
test_condition = {
    'v_ms': 20.0,
    'a_mps2': 0.0,
    'grade_pct': 0.0,
    'temp': 25,
    'temp_unit': 'C',
    'humid_pct': 50,
    'model_year': 2020
}

# Vehicle configurations to compare
vehicles = [
    {'source_type': 'Passenger Car', 'fuel_type': 'Gasoline'},
    {'source_type': 'Passenger Car', 'fuel_type': 'Diesel'},
    {'source_type': 'Passenger Truck', 'fuel_type': 'Gasoline'},
    {'source_type': 'Passenger Truck', 'fuel_type': 'Diesel'},
    {'source_type': 'Light Commercial Truck', 'fuel_type': 'Diesel'},
    {'source_type': 'Transit Bus', 'fuel_type': 'Diesel'},
]

# Calculate emissions for each vehicle
results = []
for vehicle in vehicles:
    try:
        emission = neuralmoves.estimate_running_co2(
            **test_condition,
            source_type=vehicle['source_type'],
            fuel_type=vehicle['fuel_type']
        )
        results.append({
            'Vehicle': f"{vehicle['source_type']} ({vehicle['fuel_type']})",
            'CO2 (g/s)': emission,
            'CO2 (g/km)': emission / test_condition['v_ms'] * 1000
        })
    except Exception as e:
        print(f"Error for {vehicle}: {e}")

comparison_df = pd.DataFrame(results)
comparison_df = comparison_df.sort_values('CO2 (g/s)')
print(comparison_df.to_string(index=False))

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot CO2 rate
axes[0].barh(comparison_df['Vehicle'], comparison_df['CO2 (g/s)'], color='steelblue')
axes[0].set_xlabel('CO₂ Emission Rate (g/s)', fontsize=12)
axes[0].set_title('Instantaneous Emissions at 20 m/s', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# Plot CO2 per km
axes[1].barh(comparison_df['Vehicle'], comparison_df['CO2 (g/km)'], color='coral')
axes[1].set_xlabel('CO₂ Emission (g/km)', fontsize=12)
axes[1].set_title('Distance-Normalized Emissions', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 5. Error Reporting and Transparency <a name="error-reporting"></a>

NeuralMOVES provides expected error statistics for transparent reporting:

In [ ]:
# Get overall expected error
overall_error = neuralmoves.get_expected_error()
print("Overall Model Performance:")
print(f"  MAPE (Mean Absolute Percentage Error): {overall_error['MAPE']}%")
print(f"  MPE (Mean Percentage Error): {overall_error['MPE']}%")
print(f"  MdPE (Median Percentage Error): {overall_error['MdPE']}%")
print(f"  StdPE (Std Dev of Percentage Error): {overall_error['StdPE']}%")
print(f"  MAE (Mean Absolute Error): {overall_error['MAE_g']} g/s")
print()

In [ ]:
# Get error by fuel type
print("\nError by Fuel Type:")
for fuel in ['Gasoline', 'Diesel']:
    error = neuralmoves.get_expected_error(fuel_type=fuel)
    print(f"  {fuel}: MAPE = {error['MAPE']}%, MPE = {error['MPE']}%")

In [ ]:
# Get error by vehicle type
print("\nError by Vehicle Type:")
for vtype in ['Passenger Car', 'Passenger Truck', 'Transit Bus']:
    error = neuralmoves.get_expected_error(source_type=vtype)
    if error:
        print(f"  {vtype}: MAPE = {error['MAPE']}%, MPE = {error['MPE']}%")

In [ ]:
# Get error by road grade
print("\nError by Road Grade:")
for grade in ['Zero Grade', 'Uphill', 'Downhill']:
    error = neuralmoves.get_expected_error(grade_bucket=grade)
    if error:
        print(f"  {grade}: MAPE = {error['MAPE']}%, MPE = {error['MPE']}%")

### Reporting emissions with confidence intervals

When reporting emissions, include expected error for transparency:

In [ ]:
# Calculate emission for a specific scenario
emission = neuralmoves.estimate_running_co2(
    v_ms=15.0, a_mps2=0.5, grade_pct=0.0,
    temp=25, temp_unit='C', humid_pct=50,
    model_year=2020,
    source_type='Passenger Car',
    fuel_type='Gasoline'
)

# Get expected error for this configuration
error_stats = neuralmoves.get_expected_error(
    fuel_type='Gasoline',
    source_type='Passenger Car'
)

# Report with confidence bounds
mape = float(error_stats['MAPE'])
lower_bound = emission * (1 - mape / 100)
upper_bound = emission * (1 + mape / 100)

print(f"\nEstimated Emission: {emission:.2f} g/s")
print(f"Expected Error (MAPE): ±{mape:.1f}%")
print(f"Confidence Range: [{lower_bound:.2f}, {upper_bound:.2f}] g/s")
print(f"\nFor Academic Reporting:")
print(f"  Emission = {emission:.2f} ± {(upper_bound - emission):.2f} g/s (MAPE: {mape:.1f}%)")

## 6. Real-World Scenario: Eco-Driving Analysis <a name="eco-driving"></a>

Compare aggressive vs. smooth driving styles:

In [ ]:
# Define two driving profiles covering the same distance
distance_m = 1000  # 1 km

# Aggressive driving: rapid acceleration/deceleration
aggressive_profile = pd.DataFrame({
    'speed_ms': np.concatenate([
        np.linspace(0, 25, 10),   # Fast acceleration
        [25] * 30,                 # High speed cruise
        np.linspace(25, 0, 5),     # Hard braking
        [0] * 5,                   # Stop
    ] * 3),
})

# Smooth eco-driving: gentle acceleration/deceleration
eco_profile = pd.DataFrame({
    'speed_ms': np.concatenate([
        np.linspace(0, 18, 20),    # Gentle acceleration
        [18] * 50,                 # Moderate speed cruise
        np.linspace(18, 0, 20),    # Gentle braking
    ]),
})

# Calculate accelerations and grade
for profile in [aggressive_profile, eco_profile]:
    profile['acceleration_mps2'] = np.diff(profile['speed_ms'], prepend=profile['speed_ms'].iloc[0])
    profile['grade_pct'] = 0.0

print(f"Aggressive profile: {len(aggressive_profile)} seconds")
print(f"Eco profile: {len(eco_profile)} seconds")
print(f"Aggressive distance: {aggressive_profile['speed_ms'].sum():.0f} m")
print(f"Eco distance: {eco_profile['speed_ms'].sum():.0f} m")

In [ ]:
# Calculate emissions for both profiles
def calculate_profile_emissions(profile, name):
    emissions = []
    for _, row in profile.iterrows():
        em = neuralmoves.estimate_running_co2(
            v_ms=row['speed_ms'],
            a_mps2=row['acceleration_mps2'],
            grade_pct=row['grade_pct'],
            temp=25, temp_unit='C', humid_pct=50,
            model_year=2020,
            source_type='Passenger Car',
            fuel_type='Gasoline'
        )
        emissions.append(em)
    
    profile['co2_gs'] = emissions
    profile['time_s'] = range(len(profile))
    
    total_co2 = profile['co2_gs'].sum()
    total_distance = profile['speed_ms'].sum()
    co2_per_km = total_co2 / (total_distance / 1000) if total_distance > 0 else 0
    
    print(f"\n{name}:")
    print(f"  Total CO2: {total_co2:.2f} g")
    print(f"  Distance: {total_distance:.0f} m")
    print(f"  CO2/km: {co2_per_km:.2f} g/km")
    print(f"  Time: {len(profile)} seconds")
    
    return profile

aggressive_results = calculate_profile_emissions(aggressive_profile.copy(), "Aggressive Driving")
eco_results = calculate_profile_emissions(eco_profile.copy(), "Eco-Driving")

# Calculate savings
savings_pct = (aggressive_results['co2_gs'].sum() - eco_results['co2_gs'].sum()) / aggressive_results['co2_gs'].sum() * 100
print(f"\n🌱 Eco-driving savings: {savings_pct:.1f}%")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Speed profiles
axes[0, 0].plot(aggressive_results['time_s'], aggressive_results['speed_ms'], 'r-', label='Aggressive', linewidth=1.5)
axes[0, 0].plot(eco_results['time_s'], eco_results['speed_ms'], 'g-', label='Eco-driving', linewidth=1.5)
axes[0, 0].set_ylabel('Speed (m/s)', fontsize=11)
axes[0, 0].set_title('Speed Profiles', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Acceleration profiles
axes[0, 1].plot(aggressive_results['time_s'], aggressive_results['acceleration_mps2'], 'r-', label='Aggressive', linewidth=1.5)
axes[0, 1].plot(eco_results['time_s'], eco_results['acceleration_mps2'], 'g-', label='Eco-driving', linewidth=1.5)
axes[0, 1].set_ylabel('Acceleration (m/s²)', fontsize=11)
axes[0, 1].set_title('Acceleration Profiles', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Emission rates
axes[1, 0].plot(aggressive_results['time_s'], aggressive_results['co2_gs'], 'r-', label='Aggressive', linewidth=1.5)
axes[1, 0].plot(eco_results['time_s'], eco_results['co2_gs'], 'g-', label='Eco-driving', linewidth=1.5)
axes[1, 0].set_xlabel('Time (s)', fontsize=11)
axes[1, 0].set_ylabel('CO₂ Rate (g/s)', fontsize=11)
axes[1, 0].set_title('Instantaneous Emissions', fontsize=12, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Cumulative emissions
axes[1, 1].plot(aggressive_results['time_s'], aggressive_results['co2_gs'].cumsum(), 'r-', label='Aggressive', linewidth=2)
axes[1, 1].plot(eco_results['time_s'], eco_results['co2_gs'].cumsum(), 'g-', label='Eco-driving', linewidth=2)
axes[1, 1].set_xlabel('Time (s)', fontsize=11)
axes[1, 1].set_ylabel('Cumulative CO₂ (g)', fontsize=11)
axes[1, 1].set_title('Cumulative Emissions', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Integration with Traffic Simulation <a name="traffic-simulation"></a>

Example of how to process trajectory data from traffic microsimulation:

In [ ]:
# Simulate trajectory data (as would come from SUMO, Vissim, etc.)
# Format: vehicle_id, timestep, x, y, speed, acceleration

np.random.seed(42)
n_vehicles = 3
n_timesteps = 100

trajectory_data = []
for veh_id in range(n_vehicles):
    for t in range(n_timesteps):
        # Simple trajectory with some variation
        speed = 10 + 5 * np.sin(t / 10) + np.random.normal(0, 0.5)
        speed = max(0, speed)  # No negative speeds
        
        trajectory_data.append({
            'vehicle_id': veh_id,
            'timestep': t,
            'speed_ms': speed,
        })

trajectory_df = pd.DataFrame(trajectory_data)

# Calculate accelerations per vehicle
trajectory_df['acceleration_mps2'] = trajectory_df.groupby('vehicle_id')['speed_ms'].diff().fillna(0)
trajectory_df['grade_pct'] = 0.0  # Assume flat

print(f"Loaded {n_vehicles} vehicles with {n_timesteps} timesteps each")
trajectory_df.head(10)

In [ ]:
# Calculate emissions for each trajectory point
def add_emissions_to_trajectory(df, vehicle_config):
    """Add emissions column to trajectory dataframe"""
    emissions = []
    for _, row in df.iterrows():
        em = neuralmoves.estimate_running_co2(
            v_ms=row['speed_ms'],
            a_mps2=row['acceleration_mps2'],
            grade_pct=row['grade_pct'],
            temp=25, temp_unit='C', humid_pct=50,
            **vehicle_config
        )
        emissions.append(em)
    
    df['co2_gs'] = emissions
    return df

# Assign vehicle types
vehicle_configs = [
    {'model_year': 2020, 'source_type': 'Passenger Car', 'fuel_type': 'Gasoline'},
    {'model_year': 2020, 'source_type': 'Passenger Car', 'fuel_type': 'Diesel'},
    {'model_year': 2020, 'source_type': 'Passenger Truck', 'fuel_type': 'Gasoline'},
]

# Process each vehicle
for veh_id in range(n_vehicles):
    mask = trajectory_df['vehicle_id'] == veh_id
    vehicle_df = trajectory_df[mask].copy()
    vehicle_df = add_emissions_to_trajectory(vehicle_df, vehicle_configs[veh_id])
    trajectory_df.loc[mask, 'co2_gs'] = vehicle_df['co2_gs'].values

print("Emissions calculated for all vehicles")
trajectory_df.head(10)

In [ ]:
# Aggregate emissions by vehicle and timestep
vehicle_totals = trajectory_df.groupby('vehicle_id')['co2_gs'].sum()
timestep_totals = trajectory_df.groupby('timestep')['co2_gs'].sum()

print("\nTotal emissions by vehicle:")
for veh_id, total in vehicle_totals.items():
    config = vehicle_configs[veh_id]
    print(f"  Vehicle {veh_id} ({config['source_type']}, {config['fuel_type']}): {total:.2f} g")

print(f"\nFleet total: {trajectory_df['co2_gs'].sum():.2f} g")
print(f"Average per vehicle: {vehicle_totals.mean():.2f} g")

In [ ]:
# Visualize fleet emissions over time
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Individual vehicle emissions
for veh_id in range(n_vehicles):
    veh_data = trajectory_df[trajectory_df['vehicle_id'] == veh_id]
    config = vehicle_configs[veh_id]
    label = f"Veh {veh_id}: {config['source_type']} ({config['fuel_type']})"
    axes[0].plot(veh_data['timestep'], veh_data['co2_gs'], label=label, linewidth=1.5)

axes[0].set_ylabel('CO₂ Rate (g/s)', fontsize=11)
axes[0].set_title('Individual Vehicle Emissions', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Fleet aggregate emissions
axes[1].plot(timestep_totals.index, timestep_totals.values, 'b-', linewidth=2)
axes[1].fill_between(timestep_totals.index, 0, timestep_totals.values, alpha=0.3)
axes[1].set_xlabel('Timestep (s)', fontsize=11)
axes[1].set_ylabel('Total Fleet CO₂ (g/s)', fontsize=11)
axes[1].set_title('Fleet-Wide Emissions', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook has demonstrated:

1. **Single-second estimation** - Quick emission calculations for any driving state
2. **Time-series analysis** - Processing complete driving cycles
3. **Multi-vehicle comparison** - Comparing emissions across vehicle types and fuels
4. **Error reporting** - Transparent uncertainty quantification for academic work
5. **Eco-driving analysis** - Quantifying the impact of driving behavior
6. **Traffic simulation integration** - Processing trajectory data from microsimulation

### Additional Resources

- **GitHub Repository**: https://github.com/edgar-rs/neuralMOVES
- **Paper**: [NeuralMOVES: A lightweight and microscopic vehicle emission estimation model]
- **Contact**: edgarrs@mit.edu

### Citation

If you use NeuralMOVES in your research, please cite:

```
Ramirez-Sanchez et al. (2025). NeuralMOVES: A lightweight and microscopic 
vehicle emission estimation model based on reverse engineering and surrogate learning.
Transportation Research Part C: Emerging Technologies.
```